In [2]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [6]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)  # 검증 세트

print(sub_input.shape, val_input.shape, test_input.shape)     # 훈련 세트, 검증 세트, 테스트 세트

(4157, 3) (1040, 3) (1300, 3)


In [7]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))  # 과대적합

0.9971133028626413
0.864423076923077


In [8]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)  # 5-폴드 교차 검증
print(scores)

{'fit_time': array([0.02419424, 0.02356768, 0.01267791, 0.01238561, 0.01109505]), 'score_time': array([0.003124  , 0.00478721, 0.00200558, 0.00183606, 0.00191283]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [9]:
import numpy as np

print(np.mean(scores['test_score']))    # 교차 검증의 평균 점수

0.855300214703487


In [10]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())    # 분할기 지정
print(np.mean(scores['test_score']))

0.855300214703487


In [11]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)  # 훈련 세트 섞은 후 10-폴드 교차 검증
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [14]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}    # 하이퍼파라미터로 시도할 값

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)   # 그리드 서치: 하이퍼파라미터 탐색과 교차 검증 수행
gs.fit(train_input, train_target)

dt = gs.best_estimator_     # 교차 검증 점수가 가장 높은 모델
print(dt.score(train_input, train_target))
print(gs.best_params_)      # 최적의 매개변수
print(gs.cv_results_['mean_test_score'])    # 각 매개변수의 교차 검증 점수

0.9615162593804117
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [15]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])   # 가장 큰 값의 인덱스 추출
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [18]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }     # 교차 검증할 하이퍼파라미터 범위 지정

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

print(gs.best_params_)      # 최적의 매개변수 조합
print(np.max(gs.cv_results_['mean_test_score']))    # 최적의 교차 검증 점수

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


In [19]:
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)    # 0부터 10까지 랜덤한 정수 10개 샘플링

array([2, 2, 1, 5, 2, 7, 2, 4, 7, 4])

In [20]:
np.unique(rgen.rvs(1000), return_counts=True)   # 0-10 사이 랜덤한 정수 1000개의 각 숫자의 개수

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([103, 104,  98, 113, 106, 104,  88,  90,  84, 110]))

In [21]:
ugen = uniform(0, 1)
ugen.rvs(10)    # 0부터 1까지 랜덤한 실수 10개 샘플링

array([0.97229755, 0.36266438, 0.03365055, 0.08436494, 0.03649403,
       0.03945849, 0.9038693 , 0.78785879, 0.25021146, 0.31341191])

In [23]:
from sklearn.model_selection import RandomizedSearchCV

params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }     # 랜덤 서치

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884


In [24]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))    # 테스트 세트에 대한 점수

0.86
